In [1]:
import Pkg
Pkg.add("JuMP")
Pkg.add("Gurobi")
Pkg.add("MAT")
Pkg.add("Printf")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("JLD")
Pkg.add("Plots")

using JuMP
using Gurobi
using MAT
using Printf
using DataFrames
using CSV
using Plots
using JLD

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed FFMPEG_jll ─ v4.4.2+1
  No Changes to `~/.julia/environments/v1.8/Project.toml`
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [b22a6f82] ↑ FFMPEG_jll v4.4.2+0 ⇒ v4.4.2+1
  [efcefdf7] + PCRE2_jll v10.40.0+0
Precompiling project...
  ✓ PCRE2_jll
  ✓ FFMPEG_jll
  ✓ FFMPEG
  ✓ GR_jll
  ✓ GR
  ✓ Plots
  6 dependencies successfully precompiled in 24 seconds. 172 already precompiled.
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/

In [14]:
# High-level Settings
Zone = "NORTH" # price zone name

# read RTP price
fileln = matopen(string("./RTP_",Zone,"_2019_2021_1h.mat"))
RTP = read(fileln, string("RTP_",Zone,"_2019_2021_1h"))
close(fileln)

# read DAP price
fileln = matopen(string("./DAP_",Zone,"_2019_2021.mat"))
DAP = read(fileln, string("DAP_",Zone,"_2019_2021_Julia"))
close(fileln)

# RTP = reshape(RTP,(8760,1));
# # RTP = RTP[1:24]
# DAP = reshape(DAP,(8760,1));
# # DAP = DAP[1:24];

DAP-RTP

24×365 Matrix{Float64}:
 13.3342      2.495      6.81417   …  -0.816667   -5.57583  -11.31
 12.5533      1.28583    3.88333       0.773333   -2.36417   -1.99
 10.1292      6.2975     3.5025        0.608333   -4.62917   -2.075
 18.4908      5.3075     1.65667      -0.7725     -4.50917   -4.71417
  8.83167     6.3425     8.15083      -2.28667    -6.3575    -8.38167
  7.60583    12.8792    -5.05667   …  -6.3475     -8.83208   -8.56417
  0.254167    6.96583   -5.88         -5.87667    -9.35417   -7.735
 -3.95333    14.0642    -5.60333      -6.61583    -6.1125    -6.7275
  8.0875     12.555     -5.10417      -0.598333   -4.57667   -6.58583
  3.77167    13.08      -0.423333     -0.5325     -4.7775    -7.41167
 -9.0325     12.2133    -1.14      …   2.09083     9.91917   -5.17417
 -3.5175     11.2958    11.1783        0.1425    -10.705     -2.18
 11.6975      4.6175     7.8525        1.71417    -6.30417   -1.06417
 10.4558      3.7025     3.85167       1.83333    -2.81417    3.6625
 14.8442   

In [15]:
## battery setting

E = 1;  # storage energy capacity in MWh
# simulation setting
T = 24; # total time steps
P = .5; # power rating MW
eta = 1; # efficiency
e0 = .5 * E;
ef = e0;
MC = 10; # marginal discharge cost
M = 1; # duaration per step in hour
N_sim = 365; # number of days
d_DAP = DAP[:,1]; # daily DA price
d_RTP = RTP[:,1];

# initialize optimization model
model = Model(Gurobi.Optimizer)
set_silent(model) # no outputs

@variable(model, d_da[1:T], lower_bound = 0) # discharge power in DA
# @variable(model, d_rt[1:T], lower_bound = 0) # discharge power in RT
@variable(model, c_da[1:T], lower_bound = 0) # charge power in DA
# @variable(model, c_rt[1:T], lower_bound = 0) # charge power in RT
# @variable(model, diff[1:T]) # difference between RT and DA power decision

@variable(model, e[1:T], lower_bound = 0) # total battery energy
@variable(model, R) # market revenue
# @variable(model, C) # total degradation cost
@variable(model, u_da[1:T], Bin) # 1 if charge/buy in DAM

# @constraint(model, DiffPower[t=1:T], diff[t] - d_rt[t] + c_rt[t] == -d_pDA[t] ) # actual dis/charge power 

# arbitrage revenue
@constraint(model, ArbRev, M*sum((d_DAP .- d_RTP) .* (d_da .- c_da) ) - R == 0 )
# @constraint(model, ArbRev, M*sum(RTP.* diff) - R == 0 )

# total degradation cost
# @constraint(model, DegCost, M*sum(d_rt * MC) - C == 0 )
# @constraint(model, DegCost, C == M*sum(diff .* MC))

# initial SoC evolution
@constraint(model, SoCInit, e[1] - e0 == M*(c_da[1]*eta - d_da[1]/eta) )
# rest SoC evolution
@constraint(model, SoCCont[t = 2:T], e[t] - e[t-1] == M*(c_da[t]*eta - d_da[t]/eta) )

# final energy level
@constraint(model, Enelast, e[T] >= ef )
# @constraint(model, Enelast[t = 24:24:T], e[t] >= ef )

# charging / discharging non-conflict condition
@constraint(model, ChRatingTot[t=1:T], c_da[t] <= P*u_da[t])
@constraint(model, DchRatingTot[t=1:T], d_da[t] <= P*(1-u_da[t]))

# max energy level
@constraint(model, SoCMax[t=1:T], e[t] <= E )

# maximize revenue plus degradation value
@objective(model, Max, R);
# @objective(model, Max, R);
# print(model);
# ArbRev


Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-13


In [16]:
# initialize
R_s = zeros(1, N_sim)
P_s = zeros(1, N_sim)
C_s = zeros(1, N_sim)

C_v_da = zeros(24, N_sim) # daily DA charge
D_v_da = zeros(24, N_sim) # daily DA discharge
soc = zeros(24, N_sim) # daily SoC

# step = 0

@time begin
@printf("Optimization starts...\n")

for n = 1:(N_sim)       
    # update prices
    local pda = DAP[:,n]
    local prt = RTP[:,n]

    # update prices in constraints
    for t = 1:T
        set_normalized_coefficient(ArbRev, d_da[t], M*(pda[t]-prt[t]))           
        set_normalized_coefficient(ArbRev, c_da[t], -M*(pda[t]-prt[t]))           
    end

    optimize!(model)
    
    global R_s[n] = value(R)
    # global C_s[n] = value(C)
    global P_s[n] = objective_value(model)
    # global P_s[n] = objective_value(model)

    global C_v_da[:,n] = value.(c_da)*M
    global D_v_da[:,n] = value.(d_da)*M    
    global soc[:,n] = value.(e)
    termination_status(model)
    # global e0 = value.(e)[end]
        
    @printf("Day %d, Cummulative Rev %d, Cummulative Profit %d, Cummulative Cost %d, OptStatus: %s \n", n, sum(R_s), sum(P_s), sum(C_s), termination_status(model))
    # @printf("Day %d, Cummulative Rev %d, Cummulative Profit %d, OptStatus: %s \n", n, sum(R_s), sum(P_s), termination_status(model))
 
end
    
end
rtp = vec(reshape(RTP,(8760,1)))
dap = vec(reshape(DAP,(8760,1)))

DAcharge = vec(reshape(C_v_da,(8760,1)))
DAdischarge = vec(reshape(D_v_da,(8760,1)))
SoC = vec(reshape(soc,(8760,1)))
df = DataFrame(DAP = dap, RTP = rtp, DA_Charge = DAcharge, DA_Discharge = DAdischarge, SoC = SoC)
# CSV.write("DAdispatch_918.csv", df);
CSV.write(string("DA_Dispatch_",Zone,"_2019.csv"), df);


Optimization starts...
Day 1, Cummulative Rev 46, Cummulative Profit 46, Cummulative Cost 0, OptStatus: OPTIMAL 
Day 2, Cummulative Rev 88, Cummulative Profit 88, Cummulative Cost 0, OptStatus: OPTIMAL 
Day 3, Cummulative Rev 122, Cummulative Profit 122, Cummulative Cost 0, OptStatus: OPTIMAL 
Day 4, Cummulative Rev 159, Cummulative Profit 159, Cummulative Cost 0, OptStatus: OPTIMAL 
Day 5, Cummulative Rev 186, Cummulative Profit 186, Cummulative Cost 0, OptStatus: OPTIMAL 
Day 6, Cummulative Rev 230, Cummulative Profit 230, Cummulative Cost 0, OptStatus: OPTIMAL 
Day 7, Cummulative Rev 272, Cummulative Profit 272, Cummulative Cost 0, OptStatus: OPTIMAL 
Day 8, Cummulative Rev 331, Cummulative Profit 331, Cummulative Cost 0, OptStatus: OPTIMAL 
Day 9, Cummulative Rev 394, Cummulative Profit 394, Cummulative Cost 0, OptStatus: OPTIMAL 
Day 10, Cummulative Rev 447, Cummulative Profit 447, Cummulative Cost 0, OptStatus: OPTIMAL 
Day 11, Cummulative Rev 538, Cummulative Profit 538, Cummula

In [5]:
# read DA power decision
df = CSV.read(string("./DA_Dispatch_",Zone,"_2019.csv"), DataFrame) # Desktop/2022summer/MarkovESValuation-master/DA_Dispatch_NORTH_2019.csv
dispatch = Matrix{Float64}(df);
pDA_real = -dispatch[:,3] .+ dispatch[:,4];
pDA_real

8760-element Vector{Float64}:
  0.5
  0.0
 -0.5
 -0.5
  0.5
 -0.5
  0.5
 -0.5
  0.0
  0.5
 -0.5
  0.0
  0.0
  ⋮
  0.5
  0.5
  0.0
 -0.5
  0.0
 -0.5
  0.0
  0.0
  0.5
  0.5
 -0.5
 -0.5

In [6]:
RTP = reshape(RTP,(8760,1));
DAP = reshape(DAP,(8760,1));
sum(pDA_real .* DAP)
# DAP

2035.3049999999987